In [5]:
library(factoextra)
library(FuzzyR)
library(GPArotation)
library(lattice)
library(NbClust)
library(psych)
library(pracma)
library(qlcMatrix)
library(readxl)

In [6]:
# Lê dados
dados <- read_excel("TOTAIS_SCI_MONITORAMENTO.xlsx")

In [16]:
# Índices.
dados$PERCENTUAL_IMPLEMENTADAS <-    
  dados$QTDE_RECOMENDACOES/(dados$QTDE_RECOMENDACOES-dados$CANCELADAS)
dados$PERCENTUAL_NAO_ENCAMINHADAS <-
  (dados$IMPLEMENTADAS+dados$EM_IMPLEMENTACAO)/
  (dados$QTDE_RECOMENDACOES+dados$CANCELADAS)
dados$PERCENTUAL_NAO_IMPLEMENTADAS <-
  dados$NÃO_IMPLEMENTADAS/(dados$QTDE_RECOMENDACOES-
                           dados$CANCELADAS)
dados$PERCENTUAL_CANCELADAS <-
  dados$CANCELADAS/dados$QTDE_RECOMENDACOES
dados$RECOMENDACOES_ATIVAS <-
  dados$QTDE_RECOMENDACOES-dados$CANCELADAS

head(dados)

ANO,UO,QTDE_RECOMENDACOES,IMPLEMENTADAS,NÃO_IMPLEMENTADAS,CANCELADAS,EM_IMPLEMENTACAO,A_IMPLEMENTAR,PERCENTUAL_NAO_ENCAMINHADAS,PERCENTUAL_CANCELADAS,RECOMENDACOES_ATIVAS,PERCENTUAL_IMPLEMENTADAS,PERCENTUAL_NAO_IMPLEMENTADAS
2018,SINFRA,274,120,142,12,0,0,0.41958042,0.04379562,262,1.045802,0.54198473
2017,SINFRA,242,110,97,35,0,0,0.39711191,0.14462810,207,1.169082,0.46859903
2019,SES,178,7,165,2,4,0,0.06111111,0.01123596,176,1.011364,0.93750000
2017,SES,167,26,127,6,8,0,0.19653179,0.03592814,161,1.037267,0.78881988
2019,SINFRA,137,63,52,19,3,0,0.42307692,0.13868613,118,1.161017,0.44067797
2019,SEPLAG,121,60,10,6,45,0,0.82677165,0.04958678,115,1.052174,0.08695652


In [19]:
recorte <- dados[,c(3, 5, 8, 9, 11, 13)]

In [20]:
# PCA.
PCA0 <- principal(recorte, scores=TRUE, rotate='varimax', nfactors=3)
escalonados_pca <- PCA0$scores
PCA0

Principal Components Analysis
Call: principal(r = recorte, nfactors = 3, rotate = "varimax", scores = TRUE)
Standardized loadings (pattern matrix) based upon correlation matrix
                               RC1   RC2   RC3   h2     u2 com
QTDE_RECOMENDACOES            0.98  0.04 -0.01 0.97 0.0290 1.0
NÃO_IMPLEMENTADAS             0.86  0.36 -0.02 0.87 0.1255 1.3
A_IMPLEMENTAR                -0.02  0.08  1.00 1.00 0.0032 1.0
PERCENTUAL_NAO_ENCAMINHADAS  -0.10 -0.95 -0.19 0.95 0.0514 1.1
RECOMENDACOES_ATIVAS          0.99  0.03 -0.01 0.98 0.0211 1.0
PERCENTUAL_NAO_IMPLEMENTADAS  0.14  0.97 -0.06 0.96 0.0416 1.1

                       RC1  RC2  RC3
SS loadings           2.72 1.97 1.03
Proportion Var        0.45 0.33 0.17
Cumulative Var        0.45 0.78 0.95
Proportion Explained  0.48 0.34 0.18
Cumulative Proportion 0.48 0.82 1.00

Mean item complexity =  1.1
Test of the hypothesis that 3 components are sufficient.

The root mean square of the residuals (RMSR) is  0.03 
 with the empiric

In [31]:
dados <- cbind(dados, escalonados_pca)

In [24]:
normaliza <- function(x, ...){(x - min(x, ...)) / (max(x, ...) - min(x, ...))}

In [33]:
dados$RC1n <- normaliza(dados$RC1, na.rm=TRUE)
dados$RC2n <- normaliza(dados$RC2, na.rm=TRUE)
dados$RC3n <- normaliza(dados$RC3, na.rm=TRUE)

In [45]:
head(dados)

ANO,UO,QTDE_RECOMENDACOES,IMPLEMENTADAS,NÃO_IMPLEMENTADAS,CANCELADAS,EM_IMPLEMENTACAO,A_IMPLEMENTAR,PERCENTUAL_NAO_ENCAMINHADAS,PERCENTUAL_CANCELADAS,RECOMENDACOES_ATIVAS,PERCENTUAL_IMPLEMENTADAS,PERCENTUAL_NAO_IMPLEMENTADAS,RC1,RC2,RC3,RC1n,RC2n,RC3n,escore
2018,SINFRA,274,120,142,12,0,0,0.41958042,0.04379562,262,1.045802,0.54198473,5.480447,0.1564861,-0.08279722,1.0000000,0.36543054,0.04112122,58.11998
2017,SINFRA,242,110,97,35,0,0,0.39711191,0.14462810,207,1.169082,0.46859903,4.217560,0.1257164,-0.05428097,0.8033556,0.35610072,0.04467791,48.94826
2019,SES,178,7,165,2,4,0,0.06111111,0.01123596,176,1.011364,0.93750000,3.979172,1.7991790,-0.32595470,0.7662361,0.86351916,0.01079340,63.37525
2017,SES,167,26,127,6,8,0,0.19653179,0.03592814,161,1.037267,0.78881988,3.384956,1.3036393,-0.26869290,0.6737106,0.71326426,0.01793538,54.35476
2019,SINFRA,137,63,52,19,3,0,0.42307692,0.13868613,118,1.161017,0.44067797,2.022248,0.3489844,-0.13330079,0.4615232,0.42379887,0.03482216,35.49011
2019,SEPLAG,121,60,10,6,45,0,0.82677165,0.04958678,115,1.052174,0.08695652,1.546651,-0.8805932,-0.03838689,0.3874679,0.05097285,0.04666031,20.06612


In [37]:
PCA0$Vaccounted

,RC1,RC2,RC3
SS loadings,2.7234552,1.9739920,1.0307668
Proportion Var,0.4539092,0.3289987,0.1717945
Cumulative Var,0.4539092,0.7829079,0.9547023
Proportion Explained,0.4754458,0.3446086,0.1799456
Cumulative Proportion,0.4754458,0.8200544,1.0000000


In [39]:
RC1pv <- PCA0$Vaccounted[2, 1]
RC2pv <- PCA0$Vaccounted[2, 2]
RC3pv <- PCA0$Vaccounted[2, 3]

In [44]:
dados$escore <- (RC1pv*dados$RC1n+RC2pv*dados$RC2n+RC3pv*dados$RC3n)*100

In [48]:
write.csv(dados, 'res_01.csv')